In [ ]:
# before we start BICePs sampling, let's compute J coupling constants for each state
import sys, os
sys.path.append('new_src')
from toolbox import *       
index=np.load('ind.npy')       # atom indices for each J couplings
karplus_key=np.load('Karplus.npy')    # Karplus relation for each J couplings
for i in range(100):    # 100 states in total
    print i
    J = compute_nonaa_Jcoupling('pdbs_guangfeng/%d.pdb'%i, index=index, karplus_key=karplus_key)
    np.savetxt('J_coupling/%d.txt'%i,J)


In [ ]:
# import source code
import sys, os, glob
from numpy import *
sys.path.append('new_src')
from Preparation import *
from PosteriorSampler import *
from Analysis_new import *
from Restraint import *

In [ ]:
#########################################
# Lets' create input files for BICePs
############ Preparation ################
# Specify necessary argument values
path='NOE/*txt'        # precomputed distances for each state
states=100                    # number of states
indices='atom_indice_noe.txt'   # atom indices of each distance
exp_data='noe_distance.txt'  # experimental NOE data 
top='pdbs_guangfeng/0.pdb'    # topology file 
data_dir=path                 # directory of precomputed data 
dataFiles = 'noe_J'           # output directory of BICePs formated input file from this scripts
out_dir=dataFiles               

#p=Preparation('noe',states=states,indices=indices,exp_data=exp_data,top=top,data_dir=data_dir)  # the type of data needs to be specified {'noe', 'J', 'cs_H', etc}
#p.write(out_dir=out_dir)     # raw data will be converted to a BICePs readable format to the folder specified


In [ ]:
# An example of BICePs readable format input file 
with open(dataFiles+'/0.noe', 'r') as f:
    line = f.readlines()
for i in range(len(line)):
    print line[i].strip()

In [ ]:
# similar step for J couplings
#########################################
# Lets' create input files for BICePs
############ Preparation ################
# Specify necessary argument values
path='J_coupling/*txt'        # precomputed J couplings for each state
states=100                    # number of states
indices='atom_indice_J.txt'   # atom indices of each J coupling
exp_data='exp_Jcoupling.txt'  # experimental J coupling data 
top='pdbs_guangfeng/0.pdb'    # topology file 
data_dir=path                 # directory of precomputed data 
dataFiles = 'noe_J'           # output directory of BICePs formated input file from this scripts
out_dir=dataFiles               

p=Preparation('J',states=states,indices=indices,exp_data=exp_data,top=top,data_dir=data_dir)  # the type of data needs to be specified {'noe', 'J', 'cs_H', etc}
p.write(out_dir=out_dir)     # raw data will be converted to a BICePs readable format to the folder specified


In [ ]:
# An example of BICePs readable format input file 
with open(dataFiles+'/0.J', 'r') as f:
    line = f.readlines()
for i in range(len(line)):
    print line[i].strip()

In [ ]:
#########################################
# Let's create our ensemble of structures
############ Initialization #############    
# Specify necessary argument values
data = sort_data(dataFiles)   # sorting data in the folder and figure out what types of data are used
print data
print len(data),len(data[0])
#sys.exit(1)
energies_filename =  'albocycline_QMenergies.dat'
energies = loadtxt(energies_filename)
energies = loadtxt(energies_filename)*627.509  # convert from hartrees to kcal/mol
energies = energies/0.5959   # convert to reduced free energies F = f/kT
energies -= energies.min()  # set ground state to zero, just in case
outdir = 'results_ref_normal'
# Temporarily placing the number of steps here...
nsteps = 100 # number of steps of MCMC simulation

# Make a new directory if we have to
if not os.path.exists(outdir):
    os.mkdir(outdir)


In [ ]:
######################
# Main:
######################

lambda_values = [0.0,0.5,1.0]    # lambda values to be sampled
for j in lambda_values:
    verbose = True#False
    lam = j
    # We will instantiate a number of Restraint() objects to construct the ensemble
    ensemble = []
    for i in range(energies.shape[0]):
        print '\n#### STRUCTURE %d ####'%i
        ensemble.append([])
        for k in range(len(data[0])):
            File = data[i][k]
            if verbose:
                print File

            # Call on the Restraint that corresponds to File
            if File.endswith('cs_H'):
                R = Restraint_cs_H('8690.pdb',ref='exp')
                R.prep_observable(lam=lam, free_energy=energies[i],
                        filename=File)


            elif File.endswith('cs_CA'):
                R = Restraint_cs_Ca('8690.pdb',ref='gaussian')
                R.prep_observable(lam=lam, free_energy=energies[i],
                        filename=File)

            elif File.endswith('cs_Ha'):
                R = Restraint_cs_Ha('8690.pdb',ref='exp')
                R.prep_observable(lam=lam, free_energy=energies[i],
                        filename=File)

            elif File.endswith('cs_N'):
                R = Restraint_cs_N('8690.pdb',ref='gaussian')
                R.prep_observable(lam=lam, free_energy=energies[i],
                        filename=File)

            elif File.endswith('J'):
                R = Restraint_J('pdbs_guangfeng/%d.pdb'%i,ref='uniform', dlogsigma=np.log(1.02), sigma_min=0.05, sigma_max=20.0)  # good ref
                R.prep_observable(lam=lam, free_energy=energies[i],
                        filename=File)

            elif File.endswith('noe'):
                R = Restraint_noe('pdbs_guangfeng/%d.pdb'%i,ref='exp',dlogsigma=np.log(1.02), sigma_min=0.05, sigma_max=5.0)   # good ref
                R.prep_observable(lam=lam, free_energy=energies[i],
                        filename=File, dloggamma=np.log(1.01),gamma_min=0.2,gamma_max=5.0)

            elif File.endswith('pf'):
                R = Restraint_pf('8690.pdb',ref='gaussian')
                R.prep_observable(lam=lam, free_energy=energies[i],
                        filename=File)

            ensemble[-1].append(R)
    print ensemble

    ##########################################
    # Next, let's do some posterior sampling
    ########## Posterior Sampling ############

    sampler = PosteriorSampler(ensemble)
    sampler.compile_nuisance_parameters()

    sampler.sample(nsteps)  # number of steps

    print 'Processing trajectory...',

    sampler.traj.process()  # compute averages, etc.
    print '...Done.'

    print 'Writing results...',
    sampler.traj.write_results(os.path.join(outdir,'traj_lambda%2.2f.npz'%lam))
    print '...Done.'
    sampler.traj.read_results(os.path.join(outdir,'traj_lambda%2.2f.npz'%lam))

    print 'Pickling the sampler object ...',
    outfilename = 'sampler_lambda%2.2f.pkl'%lam
    print outfilename,
    fout = open(os.path.join(outdir, outfilename), 'wb')
    # Pickle dictionary using protocol 0.
    cPickle.dump(sampler, fout)
    fout.close()
    print '...Done.'


In [ ]:
#########################################
# Let's do analysis using MBAR and plot figures
############ MBAR and Figures ###########
# Specify necessary argument values
%matplotlib inline
A = Analysis(100,dataFiles,outdir)    # number of states, input data files and output directory needs to be specified
A.plot()